In [1]:
import numpy as np
import pandas as pd
import time
import pyodbc

In [2]:
taglistfile = 'C:\\Users\meclijda\Documents\TagList.csv'
dfTags = pd.read_csv(taglistfile,";",encoding = "ISO-8859-1")
tagList = dfTags['Tag'].tolist()
print(str(len(tagList))+ ' tags in the taglist.')
#print(dfTags['Tag'].head())
#print(list(set([x for x in tagList if tagList.count(x) > 1])))

63 tags in the taglist.


In [3]:
# read IP21 tag data
strAsOfDate = '01-SEP-15 00:00:00.0'
#strAsOfDate = '01-JAN-14 00:00:00.0'
#strAsOfDate = '24-AUG-15 11:40:00.0'
strUpToDate = '19-JAN-17 00:00:00.0'
pattern = '%d-%b-%y %H:%M:%S.0'
#epochstart = int(time.mktime(time.strptime(strAsOfDate, pattern)))

start_time = time.time()

strConnect = 'DSN=PO_ODBC'
cnn = pyodbc.connect(strConnect)

#print(tagList[0])
df = pd.DataFrame()
for i, strTagName in enumerate(tagList):
    ts = time.time()
    print('{0:2}'.format(i) + ': getting data from tag ' + strTagName + ' ...')
    sql = "select TS_START, AVG from aggregates "\
        "where name = '" + strTagName + "' "\
        "and TS between '" + strAsOfDate + "' and '" + strUpToDate + "' "\
        "and period = 0:05"
    #print(sql)
    data1 = pd.read_sql(sql, cnn)
    data1['TS_START'] = pd.to_datetime(data1['TS_START'],utc=True)
    #data1['TS_START'] = data1['TS_START'].apply(lambda x: x.strftime('%Y-%m-%dT%H:%M:%SZ'))
    data1.set_index('TS_START',inplace=True)
    data1.columns = [strTagName]
    #data1.to_csv(strTagName.replace(":","_") + '.csv')
    #data2 = data1.resample('1H', how='mean')
    #print(str(data1.count())+" - "+str(data1.mean()))
    df = pd.concat([df, data1], axis=1)
    print('    data loaded in %4.1f seconds' % (time.time()-ts))

cnn.close
elapsed_time = time.time() - start_time
print('Total elapsed time: %5.1f seconds.' % elapsed_time)
#print(df.describe())

 0: getting data from tag POARFQCA-17901:me ...
    data loaded in 132.8 seconds
 1: getting data from tag POARAIA-17903_1a:av ...
    data loaded in 120.1 seconds
 2: getting data from tag POARAIA-17903_3a:av ...
    data loaded in 123.8 seconds
 3: getting data from tag POARTC-17902:me ...
    data loaded in 124.9 seconds
 4: getting data from tag POARAIA-17903_2ab:av ...
    data loaded in 83.4 seconds
 5: getting data from tag POARTI-17912:av ...
    data loaded in 122.4 seconds
 6: getting data from tag POARFCA-17907:me ...
    data loaded in 126.9 seconds
 7: getting data from tag POARTC-17923:me ...
    data loaded in 125.6 seconds
 8: getting data from tag POARTCA-17931:me ...
    data loaded in 123.4 seconds
 9: getting data from tag POARFIA-17904:av ...
    data loaded in 125.1 seconds
10: getting data from tag POARTCA-17922:me ...
    data loaded in 122.1 seconds
11: getting data from tag POARFC-17902:me ...
    data loaded in 128.2 seconds
12: getting data from tag POARTI-1

In [4]:
start_time = time.time()
df.to_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total writing time to pickle: %5.1f seconds.' % elapsed_time)

Total writing time to pickle:   0.4 seconds.


In [5]:
start_time = time.time()
df1 = pd.read_pickle('rawdata.pkl')
elapsed_time = time.time() - start_time
print('Total loading time from pickle: %5.1f seconds.' % elapsed_time)
df1.describe()

Total loading time from pickle:   0.1 seconds.


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,23.781179,69.249816,65.924474,95.424039,19.645683,103.581272,3.159821,81.515958,198.112154,130.414044,...,59.814371,16.951026,-62.037249,1.313432,1.540649,205.995559,182.690241,183.618252,166.704160,16.370524
std,2.850578,11.484184,11.433584,1.564742,39.084032,1.272847,0.131266,1.336655,1.780236,24.750951,...,3.105521,1.847151,2.491286,0.268976,0.613107,1.474026,1.615864,1.728436,1.764809,2.056762
min,3.468945,0.000000,0.000000,91.591708,0.000000,96.731805,0.909646,39.541924,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.188632,69.270771,65.406734,94.486188,0.000000,102.798947,3.096201,81.017998,196.527051,129.797351,...,58.204773,16.879801,-63.096448,1.266713,0.796764,204.862044,181.484470,182.322509,165.325283,15.885271
50%,24.663803,70.835811,68.328963,94.766015,0.000000,103.485401,3.192953,81.869529,199.003177,138.770183,...,59.745494,17.681594,-62.852466,1.359287,1.946765,205.496655,182.581187,183.467627,166.661233,17.116527
75%,25.504886,72.673524,70.292062,96.973169,0.000000,104.371185,3.219688,82.016102,199.573928,141.440047,...,61.038604,17.977719,-60.247862,1.431073,2.037982,207.299518,183.493288,184.446795,167.910096,17.658802
max,28.730017,101.736916,78.562834,130.613648,99.406669,113.543055,3.512904,87.225137,203.653258,147.169809,...,157.287056,18.915177,-50.738568,3.361272,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [6]:
df.describe()

,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,23.781179,69.249816,65.924474,95.424039,19.645683,103.581272,3.159821,81.515958,198.112154,130.414044,...,59.814371,16.951026,-62.037249,1.313432,1.540649,205.995559,182.690241,183.618252,166.704160,16.370524
std,2.850578,11.484184,11.433584,1.564742,39.084032,1.272847,0.131266,1.336655,1.780236,24.750951,...,3.105521,1.847151,2.491286,0.268976,0.613107,1.474026,1.615864,1.728436,1.764809,2.056762
min,3.468945,0.000000,0.000000,91.591708,0.000000,96.731805,0.909646,39.541924,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.188632,69.270771,65.406734,94.486188,0.000000,102.798947,3.096201,81.017998,196.527051,129.797351,...,58.204773,16.879801,-63.096448,1.266713,0.796764,204.862044,181.484470,182.322509,165.325283,15.885271
50%,24.663803,70.835811,68.328963,94.766015,0.000000,103.485401,3.192953,81.869529,199.003177,138.770183,...,59.745494,17.681594,-62.852466,1.359287,1.946765,205.496655,182.581187,183.467627,166.661233,17.116527
75%,25.504886,72.673524,70.292062,96.973169,0.000000,104.371185,3.219688,82.016102,199.573928,141.440047,...,61.038604,17.977719,-60.247862,1.431073,2.037982,207.299518,183.493288,184.446795,167.910096,17.658802
max,28.730017,101.736916,78.562834,130.613648,99.406669,113.543055,3.512904,87.225137,203.653258,147.169809,...,157.287056,18.915177,-50.738568,3.361272,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [7]:
start_time = time.time()
df.to_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total writing time to hdf5: %5.1f seconds.' % elapsed_time)

Total writing time to hdf5:   0.8 seconds.


In [8]:
start_time = time.time()
df2 = pd.read_hdf('rawdata.h5','table')
elapsed_time = time.time() - start_time
print('Total loading time from hdf5: %5.1f seconds.' % elapsed_time)
df2.describe()

Total loading time from hdf5:   0.1 seconds.


,POARFQCA-17901:me,POARAIA-17903_1a:av,POARAIA-17903_3a:av,POARTC-17902:me,POARAIA-17903_2ab:av,POARTI-17912:av,POARFCA-17907:me,POARTC-17923:me,POARTCA-17931:me,POARFIA-17904:av,...,POARTCA-17929:me,POARFCA-17219:me,POARPCA-17917:me,POARFC-17918:me,POARFC-17916:me,POARTI-17935:av,POARTI-17934:av,POARTI-17933:av,POARTI-17932:av,POARFQC-17222:me
count,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,...,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,23.781179,69.249816,65.924474,95.424039,19.645683,103.581272,3.159821,81.515958,198.112154,130.414044,...,59.814371,16.951026,-62.037249,1.313432,1.540649,205.995559,182.690241,183.618252,166.704160,16.370524
std,2.850578,11.484184,11.433584,1.564742,39.084032,1.272847,0.131266,1.336655,1.780236,24.750951,...,3.105521,1.847151,2.491286,0.268976,0.613107,1.474026,1.615864,1.728436,1.764809,2.056762
min,3.468945,0.000000,0.000000,91.591708,0.000000,96.731805,0.909646,39.541924,175.958551,0.000000,...,54.991944,0.000000,-72.407773,0.000000,0.000000,179.988764,160.192326,161.100939,145.663978,3.051103
25%,23.188632,69.270771,65.406734,94.486188,0.000000,102.798947,3.096201,81.017998,196.527051,129.797351,...,58.204773,16.879801,-63.096448,1.266713,0.796764,204.862044,181.484470,182.322509,165.325283,15.885271
50%,24.663803,70.835811,68.328963,94.766015,0.000000,103.485401,3.192953,81.869529,199.003177,138.770183,...,59.745494,17.681594,-62.852466,1.359287,1.946765,205.496655,182.581187,183.467627,166.661233,17.116527
75%,25.504886,72.673524,70.292062,96.973169,0.000000,104.371185,3.219688,82.016102,199.573928,141.440047,...,61.038604,17.977719,-60.247862,1.431073,2.037982,207.299518,183.493288,184.446795,167.910096,17.658802
max,28.730017,101.736916,78.562834,130.613648,99.406669,113.543055,3.512904,87.225137,203.653258,147.169809,...,157.287056,18.915177,-50.738568,3.361272,2.577723,216.845193,191.506107,192.079670,182.073231,19.562367


In [9]:
start_time = time.time()
df.to_csv('rawdata.csv')
elapsed_time = time.time() - start_time
print('Total writing time to csv: %5.1f seconds.' % elapsed_time)

Total writing time to csv:  29.9 seconds.
